# Library Import

In [3]:
import pandas as pd
import numpy as np
from pathlib import Path

import analyzer
import neural_network as nn
import convolutional_neural_network as cnn
import visualizer

2024-12-07 12:23:40.945964: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-07 12:23:40.988299: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-07 12:23:42.201856: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-07 12:23:42.580896: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733603022.912752 3266829 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733603023.00

# Training Data Set Preparation

In [ ]:
resolution = [250,250]
# Resolution should be consistent throughout the file

In [ ]:

# Because the training dataset are individual, the list of each data just contain themselves.
data1 = analyzer.Data("MORE_DATA/db_Y_0027.okc", resolution) 
data2 = analyzer.Data("MORE_DATA/db_Y_0030.okc", resolution)


# Training The Model

In [ ]:
# Run this section if you want to use NN model

array1, headers1, non_nan_indices1, num_grids1 = nn.data_arranger(data1.df)
array2, headers2, non_nan_indices2, num_grids2 = nn.data_arranger(data2.df)

# The learning rate, batch size, and epochs are proven to be working.

nn_model = nn.model_create_compile(headers1, 0.05)

nn_model, loss_hist = nn.model_train(nn_model, array1, 1000, 10)
nn_model, loss_hist = nn.model_train(nn_model, array2, 1000, 10)

In [ ]:
# Run this section if you want to use CNN model

array1, headers1, indices1 = cnn.data_arranger(data1.df, resolution)
array2, headers2, indices2 = cnn.data_arranger(data2.df, resolution)

# The learning rate and epochs are proven to be working.

cnn_model = cnn.model_2D_create_compile(headers1, 0.05, resolution)

cnn_model, loss_hist = cnn.model_2D_train(cnn_model, array1, 3)
cnn_model, loss_hist = cnn.model_2D_train(cnn_model, array2, 3)

# Data Classification

Choose codes from these 3 codes below to run if you want to classify few individual data.

In [ ]:
data3 = analyzer.Data("MORE_DATA/db_Y_0049.okc", resolution)

In [ ]:
# Run this code if your model choice is NN
array3, headers3, non_nan_indices3, num_grids3 = nn.data_arranger(data3.df)
data3.df = nn.model_classification(nn_model, array3, non_nan_indices3, num_grids3, data3.df, False)

In [ ]:
# Run this code if your model choice is CNN
array3, headers3, indices3 = cnn.data_arranger(data3.df, resolution)
data3.df = cnn.model_2D_classification(cnn_model, array3, indices3, data3.df, False)

Choose codes from these 3 codes below to run if you want to classify a series of data.

In [ ]:
list_paths_classify = [f"MORE_DATA/db_Y_{i:04d}.okc" for i in range(99)]

Data_classify = [analyzer.Data(path, list_paths_classify, resolution) for path in list_paths_classify] 

In [ ]:
# Run this code if your model choice is NN
for data in Data_classify:
    array, headers, non_nan_indices, num_grids = nn.data_arranger(data.df)
    data.df = nn.model_classification(nn_model, array, non_nan_indices, num_grids, data.df, False)

In [ ]:
# Run this code if your model choice is CNN
for data in Data_classify:
    array, headers, indices = cnn.data_arranger(data.df, data.resolution)
    data.df = cnn.model_2D_classification(cnn_model, array, indices, data.df, False)

# Data Export

Choose codes from these 2 codes below to run if you want to classify few individual data.

In [ ]:
# Run this code to export the image
visualizer.plot_2D_df(data3.df, 'is_boundary', 'classification.png')

In [ ]:
# Run this code to export the csv file
data3.df.to_csv('classification.csv', index=False)

Choose codes from these 2 codes below to run if you want to classify a series of data.

In [ ]:
# Run this code to export the image
for i, data in enumerate(Data_classify):
    visualizer.plot_2D_df(data.df, 'is_boundary', f'classification_{i}.png')

In [ ]:
# Run this code to export the csv file
for i, data in enumerate(Data_classify):
    data.df.to_csv(f'classification_{i}.csv', index=False)

# Temporary Code Zone
---

### Merge CSV

In [ ]:
import os
import csv

path = '3D_DATA/small_db_result'
output_file = '3D_DATA/small_db.csv'

files = [file for file in os.listdir(path) if file.endswith('.csv')]

with open(output_file, 'w', newline='', encoding='utf-8') as outfile:
    writer = None
    
    for index, filename in enumerate(files):
        print(f"merging file: {index + 1}/{len(files)}", flush=True)
        
        with open(os.path.join(path, filename), 'r', encoding='utf-8') as infile:
            reader = csv.reader(infile)
            
            header = next(reader)
            
            if writer is None:
                writer = csv.writer(outfile)
                writer.writerow(header)
            
            for row in reader:
                writer.writerow(row)

print("All files merged successfully!")


### Plot Streamline 2D

In [ ]:
from visualizer import plot_3D_to_2D_slice_streamline

plot_3D_to_2D_slice_streamline(input_file="3D_DATA/ra10e7_result/ra10e7_100.csv", output_file="streamlines.html", direction='y', seed_points_resolution=[20,20], max_time=0.2, cmap = 'viridis', axis_limits=[-0.5,0.5,0,1])

### Plot Streamline 3D

In [2]:
from visualizer import plot_3D_to_2D_slice_streamline

plot_3D_to_2D_slice_streamline(input_file="z_slice.csv", output_file="streamlines.html", direction='z', seed_points_resolution=[10,10], max_time=10, cmap = 'viridis', axis_limits=[-0.5,0.5,0.5,0.5])